In [24]:
# !pip install deepface
from deepface import DeepFace

import matplotlib.pyplot as plt
import pandas as pd 
import csv

params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)

result_df = pd.read_csv('./result/result_debface.csv')

In [25]:
result_df

,Dataset,Gender,Age,Accuracy,Total Images,Undetected
0,UTKFace,Woman,30,51.0,243,0


In [6]:
# Select a desired model
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
model_used = "ArcFace"

In [37]:
# Image selection
dataset_dir = "AFAD/AFAD-new"
# dataset_dir = "UTKFace"

# Age
age = 41

# Man, Woman
gender = "Woman"

In [33]:
def deepface_get_accuracy(dataset, age, gender, model):
    
    # set path to images based on age and gender
    path = "../../../../Monash/C2001-Y3/sem1/FIT3163/others/Assignment/Project-DebFace/DebFace/datasets"

    # AFAD
    if dataset == "AFAD/AFAD-new":
        gender_index = 0
        if gender == "Man":
            gender_index = 111
        elif gender == "Woman":
            gender_index = 112

        path = f"{path}/{dataset}/{str(age)}/{str(gender_index)}"   

    # UTKFace:
    elif dataset == "UTKFace":
        path = f"{path}/{dataset}/{gender}/{str(age)}"   

    images_array = os.listdir(path)
    total_images = len(images_array)

    res = 0
    undetected = 0
    result_analysis = []

    # Analyse each image with for loop
    for img in images_array:
        path_img = f"{path}/{img}"

        # Image analysis of age and gender 
        try:
            obj = DeepFace.analyze(img_path = path_img, actions = ['gender'], enforce_detection=False)
            result_analysis.append({'image': img, 'Gender': obj['gender']})
            if obj['gender'] == 'Man' and gender == "Man":
                res += 1
            elif obj['gender'] == 'Woman' and gender == "Woman":
                res += 1
                
        except ValueError:
            undetected += 1

    # calculate accuracy
    acc = round(res/(total_images-undetected), 2) * 100
    
    return result_analysis, {'Dataset': dataset, 'Gender': gender, 'Age':age, 'Accuracy': acc, 'Total Images': total_images, 'Undetected': undetected}

In [38]:
# Execute Deepface
result_analysis, result_acc = deepface_get_accuracy(dataset_dir, age, gender, model_used)

1/1 [==============================] - 0s 270ms/step


In [39]:
TRUTH = "Woman"

correctly_classified = []
incorrectly_classified = []

for res in result_analysis:
    if res["Gender"] == TRUTH:
        correctly_classified.append(res)
    else:
        incorrectly_classified.append(res)

print(f"Total images correctly classified: {len(correctly_classified)}\
        \nTotal images incorrectly classified: {len(incorrectly_classified)}")

Total images correctly classified: 10        
Total images incorrectly classified: 15


In [40]:
result_df_new = pd.DataFrame([result_acc])

frames = [result_df, result_df_new]
result_df = pd.concat(frames)

result_df = result_df.reset_index()
result_df.drop(columns=['index'], inplace=True)
result_df

,Dataset,Gender,Age,Accuracy,Total Images,Undetected
0,UTKFace,Woman,30,51.0,243,0
1,UTKFace,Man,30,99.0,347,0
2,AFAD/AFAD-new,Man,41,100.0,89,0
3,AFAD/AFAD-new,Woman,41,40.0,25,0


In [31]:
# Write results to csv file
result_df.to_csv("./result/result_debface.csv", index=False)